In [1]:
import time
start = time.time()

In [2]:
#IMPORTS
from datetime import datetime
from pyspark.ml.fpm import PrefixSpan
from pyspark.sql import Row
import pyspark.sql.functions as F
from math import sin, cos, sqrt, atan2, radians 
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
import operator

from pathlib import Path


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [ ]:
#parameters
support=0
support_str = "0" #used for paths
interval=15 #time window
maxDelta=3 #how many spatial delta
th=1 #distance
window_size=4 #how many time delta

In [3]:
#FILES
inputPath  = "file:///home/bigdata-01QYD/s270240/bike_sharing/filtered_status.csv"
STATION_PATH="file:///home/bigdata-01QYD/s270240/bike_sharing/station.csv"
#output files
folder_path = f'./Results_extraction/Full_Empty/Full_Empty_{interval}_{int(th*1000)}_{support_str}({window_size}-{maxDelta})'
output_file=f'{folder_path}/results_{int(th*1000)}_{support_str}_ordered_by_confidence.txt'
output_file2=f'{folder_path}/results_{int(th*1000)}_{support_str}_Vuota_Piena_ordered_by_confidence.txt'
output_file3=f'{folder_path}/results_{int(th*1000)}_{support_str}_diff_delta_ordered_by_confidence.txt'
output_file4=f'{folder_path}/results_{int(th*1000)}_{support_str}_ordered_by_support.txt'
output_file5=f'{folder_path}/results_{int(th*1000)}_{support_str}_Vuota_Piena_ordered_by_support.txt'
output_file6=f'{folder_path}/results_{int(th*1000)}_{support_str}_diff_delta_ordered_by_support.txt'
img_support=f'{window_size}-{maxDelta}-{int(th*1000)}-{support_str}.jpg'

In [4]:
inputDF = spark.read.format("csv")\
.option("delimiter", ",")\
.option("header", True)\
.option("inferSchema", True).load(inputPath)

In [5]:
inputDF=inputDF.filter("bikes_available is not null")

In [7]:
#filter for these fields
filteredDF = inputDF.filter("docks_available<>0 OR bikes_available<>0")

In [8]:
#determine if the station is full, empty or partially full
def stateFunction(docks_available, bikes_available):
    if docks_available==0:
        return 1
    elif bikes_available==0:
        return 0
    else:
        return 2

In [9]:
spark.udf.register("state", stateFunction)

<function __main__.stateFunction(docks_available, bikes_available)>

In [10]:
getStatusDF = filteredDF.selectExpr("station_id","time", "state(docks_available,bikes_available) as status")

In [11]:
# getStatusDF.show(100,False)

In [12]:
#filter only full or empty stations
full_empty=getStatusDF.filter("status==1  or status==0")

In [13]:
full_empty_count = full_empty.count()

In [14]:
full_empty_count/getStatusDF.count()*100

1.1861458605841708

In [15]:
full_empty.describe().show()

+-------+------------------+-------------------+
|summary|        station_id|             status|
+-------+------------------+-------------------+
|  count|            853763|             853763|
|   mean| 54.90071952052267|0.38197719976152633|
| stddev|19.220736591922407|0.48587127423858284|
|    min|                 2|                  0|
|    max|                84|                  1|
+-------+------------------+-------------------+



In [16]:
#create a view
full_empty.createOrReplaceTempView("readings")

In [17]:
#select station, year, month, day, hour, minute, status ordered by time
ss=spark.sql("""SELECT  station_id , YEAR(time) as year, MONTH(time) as month, DAY(time) as day, HOUR(time)as hour, MINUTE(time) as minute, status
FROM readings
GROUP BY station_id, year, month, day,hour,minute, status
ORDER BY  station_id,year, month,day, hour,minute
""")

In [18]:
#create rdd and group into interval
my_rdd=ss.rdd.map(tuple)
rdd=my_rdd.map(lambda line: (line[0],line[1],line[2], line[3], line[4], int(line[5]/interval), line[6])).distinct()
# rdd.collect()

# [(22, 2015, 6, 19, 7, 0, '0'),
#  (22, 2015, 6, 19, 12, 1, '0'),
#  (41, 2014, 4, 18, 13, 1, '0'),
#  (47, 2013, 11, 7, 6, 1, '0'),
#  (50, 2014, 11, 4, 8, 0, '1'),
#  (50, 2015, 1, 13, 6, 0, '1'),
#  (50, 2015, 1, 20, 18, 1, '1'),
#  (56, 2015, 1, 31, 21, 1, '1'),
#  (56, 2015, 5, 21, 16, 0, '0'), ...]

In [19]:
#get distinct stations to calculate distances
id_stations=rdd.map(lambda line: line[0]).distinct()

In [20]:
tot_id_stations=id_stations.collect()

In [21]:
#list of all stations
# tot_id_stations

In [22]:
#obtain timestamp and info
def getMap(line):
    id_station=str(line[0])
    year=int(line[1])
    month=int(line[2])
    day=int(line[3])
    hour=int(line[4])
    minute=int(line[5])   
    timestamp= datetime(year,month, day, hour, minute)  
    status=int(line[6])
    if status==0:
        status='Vuota'
    else:
        status='Piena'
    info=id_station.split('.')[0]+'_'+status
    return ( (timestamp,info))

In [23]:
get_map=rdd.map(getMap)
# get_map.collect()

# [(datetime.datetime(2015, 6, 19, 7, 0), '22_Vuota'),
#  (datetime.datetime(2015, 6, 19, 12, 1), '22_Vuota'),
#  (datetime.datetime(2014, 4, 18, 13, 1), '41_Vuota'),
#  (datetime.datetime(2013, 11, 7, 6, 1), '47_Vuota'), ...]


In [24]:
#for each timestamp obtain info
reduceK=get_map.reduceByKey(lambda l1,l2 :(l1+','+l2)).sortByKey()
# reduceK.collect()

# [(datetime.datetime(2013, 8, 29, 10, 0), '67_Vuota,70_Vuota,7_Vuota,4_Vuota,13_Vuota'),
#  (datetime.datetime(2013, 8, 29, 10, 1), '7_Vuota,4_Vuota,67_Vuota,13_Vuota,45_Piena'),
#  (datetime.datetime(2013, 8, 29, 11, 0), '4_Vuota,45_Piena,67_Vuota,64_Piena'),
#  (datetime.datetime(2013, 8, 29, 12, 0), '69_Vuota'),
#  (datetime.datetime(2013, 8, 29, 12, 1), '64_Piena,69_Vuota'),
#  (datetime.datetime(2013, 8, 29, 13, 0), '69_Vuota,76_Vuota,64_Piena'), ...]


In [25]:
my_df=reduceK.toDF()
# my_df.collect()

# [Row(_1=datetime.datetime(2013, 8, 29, 8, 0), _2='67_Vuota,70_Vuota,7_Vuota,4_Vuota,13_Vuota'),
#  Row(_1=datetime.datetime(2013, 8, 29, 8, 1), _2='7_Vuota,4_Vuota,67_Vuota,13_Vuota,45_Piena'),
#  Row(_1=datetime.datetime(2013, 8, 29, 9, 0), _2='4_Vuota,45_Piena,67_Vuota,64_Piena'),
#  Row(_1=datetime.datetime(2013, 8, 29, 10, 0), _2='69_Vuota'),
#  Row(_1=datetime.datetime(2013, 8, 29, 10, 1), _2='64_Piena,69_Vuota'), ...]

In [26]:
my_df.createOrReplaceTempView("view")

In [27]:
s2=spark.sql("""SELECT ROW_NUMBER() OVER(ORDER BY _1,_2) as id ,_1, _2
FROM view """)

In [28]:
#identifier of the timestamp, info
rdd_scheme=s2.rdd.map(tuple).map(lambda line: (line[0], line[2]))
# rdd_scheme.collect()

# [(1, '67_Vuota,70_Vuota,7_Vuota,4_Vuota,13_Vuota'),
#  (2, '7_Vuota,4_Vuota,67_Vuota,13_Vuota,45_Piena'),
#  (3, '4_Vuota,45_Piena,67_Vuota,64_Piena'),
#  (4, '69_Vuota'),
#  (5, '64_Piena,69_Vuota'),
#  (6, '69_Vuota,76_Vuota,64_Piena'), ... ]


In [29]:
#obtain window, station-status
def giveSplit(line):   
    id_window=( int(line[0] ))
    lista=[]    
    counter=id_window    
    while counter>=1:
        lista.append(('Window '+str(counter),(line[1])))
        counter=counter-1
        if (id_window-counter)==window_size:
            return lista  
    return lista

In [30]:
mapData=rdd_scheme.flatMap(giveSplit)
# mapData.collect()

# [('Window 1', '67_Vuota,70_Vuota,7_Vuota,4_Vuota,13_Vuota'),
#  ('Window 2', '7_Vuota,4_Vuota,67_Vuota,13_Vuota,45_Piena'),
#  ('Window 1', '7_Vuota,4_Vuota,67_Vuota,13_Vuota,45_Piena'),
#  ('Window 3', '4_Vuota,45_Piena,67_Vuota,64_Piena'),
#  ('Window 2', '4_Vuota,45_Piena,67_Vuota,64_Piena'),
#  ('Window 1', '4_Vuota,45_Piena,67_Vuota,64_Piena'),... ]

In [31]:
# for each window get all info
all_keys=mapData.reduceByKey(lambda l1,l2:(l1+'-'+l2))

In [32]:
# all_keys.collect()

# [('Window 1', '67_Vuota,70_Vuota,7_Vuota,4_Vuota,13_Vuota-7_Vuota,4_Vuota,67_Vuota,13_Vuota,45_Piena-4_Vuota,45_Piena,67_Vuota,64_Piena'),
#  ('Window 2', '7_Vuota,4_Vuota,67_Vuota,13_Vuota,45_Piena-4_Vuota,45_Piena,67_Vuota,64_Piena-69_Vuota'),
#  ('Window 3', '4_Vuota,45_Piena,67_Vuota,64_Piena-69_Vuota-64_Piena,69_Vuota'),
#  ('Window 4', '69_Vuota-64_Piena,69_Vuota-69_Vuota,76_Vuota,64_Piena'),
#  ('Window 5', '64_Piena,69_Vuota-69_Vuota,76_Vuota,64_Piena-69_Vuota,76_Vuota'), ...]

In [33]:
#finestra temporale
def reduceKeys(line):   
    lista=[]
    #lista.append(line[0])
    line_split=line[1].split("-")
    #return line_split[0]
    count=len(line_split)
    tot=[]
    for val in range(count):
        li=[]
        stations=line_split[val].split(',')
        for st in stations:
            all_string_st=st.split('_')[0]+'_'+'T'+str(val)+'_'+st.split('_')[1]
            li.append(all_string_st)
        tot.append(li)
    lista.append((line[0],(tot))) 
    return lista

In [34]:
windows=all_keys.flatMap(reduceKeys)

In [35]:
# windows.collect()

# [('Window 1',
#   [['67_T0_Vuota', '70_T0_Vuota', '7_T0_Vuota', '4_T0_Vuota', '13_T0_Vuota'],
#    ['7_T1_Vuota', '4_T1_Vuota', '67_T1_Vuota', '13_T1_Vuota', '45_T1_Piena'],
#    ['4_T2_Vuota', '45_T2_Piena', '67_T2_Vuota', '64_T2_Piena']]),
#  ('Window 2',
#   [['7_T0_Vuota', '4_T0_Vuota', '67_T0_Vuota', '13_T0_Vuota', '45_T0_Piena'],
#    ['4_T1_Vuota', '45_T1_Piena', '67_T1_Vuota', '64_T1_Piena'],
#    ['69_T2_Vuota']]), ... ]

In [36]:
#save station file
stationsDF = spark.read.format("csv")\
.option("delimiter", ",")\
.option("header", True)\
.option("inferSchema", True).load(STATION_PATH)

In [37]:
#get only rows interested: only the used stations 
necessary_rows=stationsDF.filter(F.col("id").isin(tot_id_stations)).sort("id").rdd.map(tuple)

In [38]:
# necessary_rows.collect()

In [39]:
#get info of stations about coordinates and name
coordinates=necessary_rows.map(lambda line: (line[0],(str(line[2])+','+str(line[3]))))
names_stations=necessary_rows.map(lambda line: (line[0],line[1]))

In [40]:
list_coo=coordinates.collect()
# list_coo

# [(2, '37.329732,-121.90178200000001'),
#  (3, '37.330698,-121.888979'),
#  (4, '37.333988,-121.894902'),... ]

In [41]:
#create a dictionary in which the key is the station and value is the info about coordinates
dic_co=coordinates.collectAsMap()
dic_coordinates=sc.broadcast(dic_co)
# dic_coordinates.value

In [42]:
#function to retrieve distance between 2 stations
def getDistance(station1,station2):
    # approximate radius of earth in km
    R = 6373.0    
    lat_a=float(station1.split(',')[0])
    lat_b=float(station2.split(',')[0])
    long_a=float(station1.split(',')[1])
    long_b=float(station2.split(',')[1])
    
    lat1=radians(lat_a)
    lat2=radians(lat_b)
    lon1=radians(long_a)
    lon2=radians(long_b)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [43]:
#voc in which the key is a pair of stations and value is the distance
voc_distances={}
for i in range(len(list_coo)):
    for j in range(i+1,len(list_coo)):
        station1=list_coo[i][0]
        station2=list_coo[j][0]
        d_i=list_coo[i][1]
        d_j=list_coo[j][1]
        distance=getDistance(d_i,d_j)
        id_stations=str(station1)+' '+str(station2)
        voc_distances[id_stations]=distance

In [44]:
voc_distances['2 3']

1.1374454650255135

In [45]:
# voc_distances

In [46]:
#Applicazione “Delta Spaziale”
def giveSpatialWindow(line):
    lista=[]    
    time0=line[1][0]    
    dic={}
    
    count_windows=len(line[1])#tot windows

    for station in time0:# only first window
        act_station=int(station.split('_')[0])
        #lista_station=[] 
        list_tmp=[]
        
        #for each window
        for i,window in enumerate(line[1]):           
            second_lista=[]
            #for each element of a window
            for all_el in window :
                #second_lista=[]
                act_all_el=int(all_el.split('_')[0])
                state=all_el.split('_')[2]
               
                if act_station!=act_all_el:
                    
                    key=''
                    if act_station<act_all_el:
                        key=str(act_station)+' '+str(act_all_el)
                    else:
                        key=str(act_all_el)+' '+str(act_station)                    
                    
                    dist=voc_distances[key]
                    if dist<=maxDelta*th:
                        delta=0
                        for d in range(1,maxDelta+1):
                            if d*th>=dist:
                                delta=d
                                break                        
                        string=state+'_'+'T'+str(i)+'_'+str(delta)
                        second_lista.append(string)
                else:
                    string=state+'_'+'T'+str(i)+'_'+str(0)
                    second_lista.append(string)
                    
            if len(second_lista)>0:
                list_tmp.append(second_lista)
        lista.append(((line[0]+'|'+str(act_station)),list_tmp))
    
    return lista

In [47]:
spatial_app=windows.flatMap(giveSpatialWindow)

In [48]:
# spatial_app.collect()

# [('Window 1|67',
#   [['Vuota_T0_0', 'Vuota_T0_2'],
#    ['Vuota_T1_0', 'Piena_T1_3'],
#    ['Piena_T2_3', 'Vuota_T2_0', 'Piena_T2_3']]),
#  ('Window 1|70',
#   [['Vuota_T0_2', 'Vuota_T0_0'],
#    ['Vuota_T1_2', 'Piena_T1_3'],
#    ['Piena_T2_3', 'Vuota_T2_2', 'Piena_T2_1']]),


In [49]:
def row_seq(line):
    true=line[1]
    string=Row(sequence=true)
    return string

In [50]:
spatial=spatial_app.map(row_seq)

In [51]:
#create a dataframe
df=spatial.toDF()

In [52]:
supports=[support]

In [53]:
df.show(20,False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------+
|sequence                                                                                                                                                  |
+----------------------------------------------------------------------------------------------------------------------------------------------------------+
|[[Vuota_T0_0, Vuota_T0_1, Vuota_T0_1], [Vuota_T1_0, Vuota_T1_1, Vuota_T1_1], [Vuota_T2_1, Vuota_T2_0, Vuota_T2_1], [Vuota_T3_0, Vuota_T3_1], [Vuota_T4_0]]|
|[[Vuota_T0_1, Vuota_T0_0, Vuota_T0_1], [Vuota_T1_1, Vuota_T1_1, Vuota_T1_0], [Vuota_T2_1, Vuota_T2_1, Vuota_T2_0], [Vuota_T3_1, Vuota_T3_1], [Vuota_T4_1]]|
|[[Vuota_T0_1, Vuota_T0_1, Vuota_T0_0], [Vuota_T1_1, Vuota_T1_0, Vuota_T1_1], [Vuota_T2_0, Vuota_T2_1, Vuota_T2_1], [Vuota_T3_1, Vuota_T3_0], [Vuota_T4_1]]|
|[[Vuota_T0_0], [Vuota_T1_0, Vuota_T1_2], [Vuota_T2_2, Pie

In [ ]:
#prefixspan to obtain sequence and frequence
counter=[]
for support in supports:
    print(support)
    prefixSpan = PrefixSpan(minSupport=support, maxPatternLength=5,
                        maxLocalProjDBSize=5000)
    prefix=prefixSpan.findFrequentSequentialPatterns(df)   
    len_prefix=prefix.count()    
    prefix.show(len_prefix,False)
    counter.append(len_prefix)

In [ ]:
len_prefix

In [ ]:
pre=prefix.rdd.map(tuple)

In [ ]:
# pre.collect()

# [([['Vuota_T1_1']], 18487),
#  ([['Piena_T2_1']], 10252),
#  ([['Piena_T1_2']], 14445), ...]


In [ ]:
def giveSelected(line):
    seq=line[0]
    found=False   
    for window in seq:
        for el in window:
            if 'T0' in el and '_0' in el:
                found=True
                break
    return found 

In [ ]:
giveT0=pre.filter(giveSelected)

In [ ]:
# giveT0.collect()

In [ ]:
df2=giveT0.toDF().withColumnRenamed('_1','sequence')
df2=df2.withColumnRenamed('_2','freq')#.show(len_prefix,False)

In [ ]:
def mapValues(line):
    seq=line[0]
    final=''
    #voc[seq]=line[1]
    for i,window in enumerate(seq):
        if i>0:
            final+='-'
        for j,el in enumerate(window):
            if j>0:
                final+=','
            final+=el
    final+=(';'+str(line[1])+';'+str(i))
    return final  

In [ ]:
mapDict=giveT0.map(mapValues)

In [ ]:
# mapDict.collect()

# ['Piena_T0_0;19530;0',
#  'Vuota_T0_0;31245;0',
#  'Vuota_T0_0-Vuota_T1_0;19880;1',
#  'Vuota_T0_0,Piena_T0_0;76;0',...
#  'Vuota_T0_0,Vuota_T0_2,Piena_T0_3;3109;0',
#  'Piena_T0_0,Vuota_T0_3-Vuota_T2_3;2451;1', ...]

In [ ]:
li=mapDict.collect()
voc={}

In [ ]:
for el in li:
    splits=el.split(';')
    voc[splits[0]]=int(splits[1])

In [ ]:
#voc

In [ ]:
repeated_el_window=0
for el in voc.keys():
    flag_rep=False
    windows=el.split('-')
    for w in windows:
        tot_items=len(w.split(','))
        set_items=len(set(w.split(',')))
        if tot_items!=set_items:
            repeated_el_window+=1
            break
repeated_el_window

In [ ]:
voc_supports={}
for el in voc.keys():    
    if len(el.split('-'))>1:        
        num=int(voc[el])       
        string=''
        tot=el.split('-')[:-1]
        for k,station in enumerate(tot):
            if k>0:
                string+='-'
            string+=station
        #print(string)
        den=int(voc[string])
        #nella seguente riga viene riportato supporto - supportCount
        voc_supports[el]=str(num/den)+' - '+str(voc[el])
keys=list(voc_supports.keys())
values=list(voc_supports.values())

In [ ]:
#sort vocabulary by decreasing values and sort within each window
for key in voc_supports:    
    splitted=key.split('-')      
    splitted.sort()
# voc_supports = dict(sorted(voc_supports.items(), key=operator.itemgetter(1),reverse=True))
voc_supports = dict(sorted(voc_supports.items(), 
                           key=lambda v: (float(v[1].split(' - ')[0]), int(v[1].split(' - ')[1])),
                           reverse=True))

In [ ]:
len(voc_supports)

In [ ]:
# output_file='results_..._ordered_by_confidence.txt'
file = open(output_file, "w")

In [ ]:
list_pattern=[]

In [ ]:
for el in voc_supports:
    key_list=[]
    for e in el.split('-'):
        key_list.append([e])
    list_pattern.append([[key_list], [voc_supports[el]]])

In [ ]:
file.write('Pattern, Confidence-Frequence'+'\n')
file.write(f'Total number of input patterns: {len(voc_supports)}'+'\n')
for el in list_pattern:  
    file.write(str(el)+ '\n')    
file.close() 

In [ ]:
# output_file4='results_..._ordered_by_support.txt'
file4 = open(output_file4, "w")

In [ ]:
# list_pattern[1][1][0].split(" - ")[1]
# list_pattern[1][0][0]

In [ ]:
#order list fist by support and then by confidence
list_ordered_by_support = sorted(list_pattern,
                                 key=lambda v: (int(v[1][0].split(" - ")[1]), float(v[1][0].split(" - ")[0])),
                                 reverse=True)

In [ ]:
file4.write('Pattern, Confidence-Frequence'+'\n')
file4.write(f'Total number of input patterns: {len(voc_supports)}'+'\n')
for el in list_ordered_by_support:  
    file4.write(str(el)+ '\n')    
file4.close() 

In [ ]:
tot_list=[]
for i, j in zip(keys,values):
    key_list=[]
    for el in i.split('-'):
        key_list.append([el])
    tot_list.append([[key_list], [j]])

In [ ]:
# list1 = [1,2,3]
# list2 = ["a","b","c"]
# for i, j in zip(list1 ,list2):
#     print (f"i: {i}, j: {j}")
    

In [ ]:
confidence=[]
# print(values)
for el in values:
#     if "e" in el:
#         v= el.split('-')[0] +"-"+ el.split('-')[1]
#         val=round(float(v),2)
#     else:
    val=round(float(el.split(' - ')[0]),2)
    confidence.append(val)
#confidence    

In [ ]:
plt.hist(confidence)
plt.xlabel('Confidence')
plt.ylabel('Number of patterns')
plt.title('3 spatial thresholds, 3 time thresholds, 1000 meters, support=0')
plt.xlim(0,1)
plt.savefig(img_support)
plt.show()

In [ ]:
#how many min, max, avg items there are
num_items=0
num_freq_items=0
min_items=sys.maxsize
max_items=0
avg_items=0

num_freq_window=0
tot_window=0
num_patterns=0
min_window=sys.maxsize
max_window=0
avg_window=0
flag_piena=False
flag_quasi_piena=False

list_piena_quasi_piena=[]

for key in voc_supports.keys():
    flag_piena=False
    flag_quasi_piena=False    
    string=''
    
    #statistics for the windows
    if '-' in key:
        patterns=len(key.split('-'))
        tot_window+=patterns      
    else:
        tot_window+=1
        patterns=1
    num_patterns+=1
    
    #statistics for the ìtems
    if ('-' in key and ',' in key):
        new_key=key       
        new_key=new_key.replace(',', '-')               
        items=new_key.split('-')
        for el in items:
            string+=el.split('_')[0]
            if el.split('_')[0].startswith('Piena') and flag_piena==False:
                flag_piena=True
            elif el.split('_')[0].startswith('Vuota') and flag_quasi_piena==False:
                flag_quasi_piena=True
      
       
    elif ('-' in key):
        items=key.split('-')
        for el in items:
            string+=el.split('_')[0]
            if el.split('_')[0].startswith('Piena') and flag_piena==False:
                flag_piena=True
            elif el.split('_')[0].startswith('Vuota') and flag_quasi_piena==False:
                flag_quasi_piena=True
       
    elif ( ',' in key):
        items=key.split(',')
        for el in items:
            string+=el.split('_')[0]
            if el.split('_')[0].startswith('Piena') and flag_piena==False:
                flag_piena=True
            elif el.split('_')[0].startswith('Vuota') and flag_quasi_piena==False:
                flag_quasi_piena=True
      
    else:
        items=list(key)
        for el in items:
            string+=el.split('_')[0]
            if el.split('_')[0].startswith('Piena') and flag_piena==False:
                flag_piena=True
            elif el.split('_')[0].startswith('Vuota') and flag_quasi_piena==False:
                flag_quasi_piena=True
    if (flag_quasi_piena==True and flag_piena==True):       
        #list_piena_quasi_piena.append([key])
        key_list=[]            
        for e in key.split('-'):
            key_list.append([e])
        list_piena_quasi_piena.append([[key_list], [voc_supports[key]]])

    q_items=len(items) 
    if min_items>q_items:
        min_items=q_items
    if max_items<q_items:
        max_items=q_items
    freq=int(voc_supports[key].split('-')[1])
    num_items+=freq 
    num_freq_items+=freq*q_items
    
    if min_window> patterns:
        min_window=patterns
    if max_window<patterns:
        max_window=patterns    
    num_freq_window+=freq*patterns  

avg_window=0
if num_items!=0:
    avg_items=float(num_freq_items)/float(num_items)    
    print(f'The average number of items is: {avg_items}')    
    avg_window=float(num_freq_window)/ float(num_items)
    print(f'The average number of windows is: {avg_window}')
else:
    print(f'The average number of windows is: {avg_window}')
    
print(f'The minimum number of items is: {min_items}')
print(f'The maximum number of items is: {max_items}')
print(f'The minimum number of windows is: {min_window}')
print(f'The maximum number of windows is: {max_window}')

In [ ]:
print('FILTER PATTERNS WITH AT LEAST 1 EVENT VUOTA AND 1 EVENT PIENA ')

In [ ]:
# output_file2='results_..._Vuota_Piena_ordered_by_confidence.txt'
file2 = open(output_file2, "w")

In [ ]:
# lung_piena_quasipiena=0
if len(list_piena_quasi_piena)!=0:
    df_supports=sc.parallelize(list_piena_quasi_piena).toDF().withColumnRenamed('_1','sequence')
    df_supports=df_supports.withColumnRenamed('_2','confidence-freq')
    lung_piena_quasipiena=df_supports.count() 
#     df_supports.show(lung_piena_quasipiena,False)
#     print(lung_piena_quasipiena)

In [ ]:
#list_piena_quasi_piena

In [ ]:
file2.write('Pattern, Confidence-Frequence'+'\n')
file2.write(f'Total number of input patterns: {len(list_piena_quasi_piena)}'+'\n')
if len(list_piena_quasi_piena)!=0:
    for el in list_piena_quasi_piena:
        #print(el)
        file2.write(str(el)+'\n')
file2.close()

In [ ]:
# output_file5='results_..._Vuota_Piena_ordered_by_support.txt'
file5 = open(output_file5, "w")

In [ ]:
#order list fist by support an then by confidence
list_piena_quasi_piena_ordered_by_support = sorted(list_piena_quasi_piena,
                                                   key=lambda v: (int(v[1][0].split(" - ")[1]), float(v[1][0].split(" - ")[0])),
                                                   reverse=True)

In [ ]:
file5.write('Pattern, Confidence-Frequence'+'\n')
file5.write(f'Total number of input patterns: {len(list_piena_quasi_piena_ordered_by_support)}'+'\n')
if len(list_piena_quasi_piena_ordered_by_support)!=0:
    for el in list_piena_quasi_piena_ordered_by_support:
        #print(el)
        file5.write(str(el)+'\n')
file5.close()

In [ ]:
print('FILTER PATTERNS WITH AT LEAST 1 T0, DELTA S=0 AND AT LEAST 1 PATTERN WITH AT LEAST 1 PATTERN WITH DELTA S DIFFERENT FROM 0 AND DELTA T DIFFERENT FROM 0')

In [ ]:
list_influenze=[]
for el in voc_supports.keys():
    delta_spaziale=False
    if ('-' in el):
        all_windows_list=el.split('-')
        if ('T0_0' in all_windows_list[0] ):
            for cons_window in all_windows_list[1::]:
                if ',' in cons_window:
                    for item in cons_window.split(','):                       
                        act_delta=int(item.split('_')[2])
                        if act_delta!=0:
                            delta_spaziale=True
    if delta_spaziale==True:        
        key_list=[]            
        for e in el.split('-'):
            key_list.append([e])
        list_influenze.append([[key_list], [voc_supports[el]]])

In [ ]:
# output_file3='results_..._diff_delta_ordered_by_confidence.txt'
file3 = open(output_file3, "w")

In [ ]:
lung_different_time_space=0
if len(list_influenze)!=0:
    df_supports=sc.parallelize(list_influenze).toDF().withColumnRenamed('_1','sequence')
    df_supports=df_supports.withColumnRenamed('_2','confidence-freq')
    lung_different_time_space=df_supports.count() 
    #df_supports.show(lung_different_time_space,False)
    #print(lung_different_time_space)

In [ ]:
file3.write('Pattern, Confidence-Frequence'+'\n')
file3.write(f'Total number of input patterns: {len(list_influenze)}'+'\n')
if len(list_influenze)!=0:
    for el in list_influenze:       
        file3.write(str(el)+'\n')
file3.close()

In [ ]:
# output_file6='results_..._diff_delta_ordered_by_support.txt'
file6 = open(output_file6, "w")

In [ ]:
#order list first by support and then by confidence
list_influenze_ordered_by_support = sorted(list_influenze,
                                           key=lambda v: (int(v[1][0].split(" - ")[1]), float(v[1][0].split(" - ")[0])),
                                           reverse=True)

In [ ]:
file6.write('Pattern, Confidence-Frequence'+'\n')
file6.write(f'Total number of input patterns: {len(list_influenze_ordered_by_support)}'+'\n')
if len(list_influenze_ordered_by_support)!=0:
    for el in list_influenze_ordered_by_support:       
        file6.write(str(el)+'\n')
file6.close()

In [ ]:
print(f'The number of patterns in the pre-filter is: {len_prefix}')

In [ ]:
print(f'The number of items after the filter with at least 2 windows and at least a T0 and delta 0 is: {len(voc_supports)}')

In [ ]:
print('STATISTICS about sequences with at least 2 windows_T0_delta0')
print(f'The average number of windows is: {avg_window}')
print(f'The minimum number of windows is: {min_window}')
print(f'The maximum number of windows is: {max_window}')
print(f'The average number of items is: {avg_items}')
print(f'The minimum number of items is: {min_items}')
print(f'The maximum number of items is: {max_items}')

In [ ]:
print(f'The number of patterns in which there is at least one item that repeats within a window is: {repeated_el_window} ')

In [ ]:
print(f'The number of patterns with at least 1 event QuasiPiena and 1 event Piena is: {len(list_piena_quasi_piena)}')

In [ ]:
 print(f'The number of patterns with at least 1 T0, DELTA S=0 and at least 1 pattern with at least 1 pattern with DELTA S different from 0 and DELTA T different from 0 is: {lung_different_time_space}')

In [ ]:
end = time.time()
print(f'The time of execution is: {end-start} s')